In [1]:
import pandas as pd
import csv
import glob
import plotly.graph_objects as go
import plotly_express as px

In [2]:
### Column headers on the public_lar file data for 2018. Definitions: https://ffiec.cfpb.gov/documentation/2018/lar-data-fields/

names = ['activity_year', 'lei', 'derived_msa_md', 'state_code', 'county_code', 'census_tract', 'conforming_loan_limit', 'derived_loan_product_type', 'derived_dwelling_category', 'derived_ethnicity', 'derived_race', 'derived_sex', 'action_taken', 'purchaser_type', 'preapproval', 'loan_type', 'loan_purpose', 'lien_status', 'reverse_mortgage', 'open_end_line_of_credit', 'business_or_commercial_purpose', 'loan_amount', 'loan_to_value_ratio', 'interest_rate', 'rate_spread', 'hoepa_status', 'total_loan_costs', 'total_points_and_fees', 'origination_charges', 'discount_points', 'lender_credits', 'loan_term', 'prepayment_penalty_term', 'intro_rate_period', 'negative_amortization', 'interest_only_payment', 'balloon_payment', 'other_nonamortizing_features', 'property_value', 'construction_method', 'occupancy_type', 'manufactured_home_secured_property_type', 'manufactured_home_land_property_interest', 'total_units', 'multifamily_affordable_units', 'income', 'debt_to_income_ratio', 'applicant_credit_score_type', 'co_applicant_credit_score_type', 'applicant_ethnicity_1', 'applicant_ethnicity_2', 'applicant_ethnicity_3', 'applicant_ethnicity_4', 'applicant_ethnicity_5', 'co_applicant_ethnicity_1', 'co_applicant_ethnicity_2', 'co_applicant_ethnicity_3', 'co_applicant_ethnicity_4', 'co_applicant_ethnicity_5', 'applicant_ethnicity_observed', 'co_applicant_ethnicity_observed', 'applicant_race_1', 'applicant_race_2', 'applicant_race_3', 'applicant_race_4', 'applicant_race_5', 'co_applicant_race_1', 'co_applicant_race_2', 'co_applicant_race_3', 'co_applicant_race_4', 'co_applicant_race_5', 'applicant_race_observed', 'co_applicant_race_observed', 'applicant_sex', 'co_applicant_sex', 'applicant_sex_observed', 'co_applicant_sex_observed', 'applicant_age', 'co_applicant_age', 'applicant_age_above_62', 'co_applicant_age_above_62', 'submission_of_application', 'initially_payable_to_institution', 'aus_1', 'aus_2', 'aus_3', 'aus_4', 'aus_5', 'denial_reason_1', 'denial_reason_2', 'denial_reason_3', 'denial_reason_4', 'tract_population', 'tract_minority_population_percent', 'ffiec_msa_md_median_family_income', 'tract_to_msa_income_percentage', 'tract_owner_occupied_units', 'tract_one_to_four_family_homes', 'tract_median_age_of_housing_units']

In [3]:
### Setting variables for what we might want to filter by

mo = ['MO']
il = ['IL']
stl = ['41180']
stl_c = [29189]
stl_city = [29510]
sf = ['Single Family (1-4 Units):Site-Built', 'Single Family (1-4 Units):Manufactured']
sf_old = [1]
orig = [1]
purchase = [1]
refi = [31]
refi_old = [3]
impr = [2]
cash_refi = [32]
other_purp = [4]
black_old = ['Black or African American']
black_new = [3]
white_old = ['White']
white_new = [5]

In [148]:
### Assigning bank names to LEIs for 2018 data

banks = {'6BYL5QZYBDK8S7L73M02': 'US Bank', 'KB1H1DSPRFMYMCUFXT09': 'Wells Fargo', 'B4TYDEB6GKMZO031MB27': 'BOA', '7H6GLXDRUGQFU57RNE97': 'JP Morgan/Chase', '549300T67186NPGCHI50': 'First Community', '549300FGXN1K3HLB1R50': 'Quicken', '549300AQ3T62GXDU7D76': 'Guild Mortgage Company', '549300ZZME37MXI1EF14': 'DAS Acquisition', '7DMUJTL9FFTVIAG9H788': 'Commerce Bank', '549300LYRWPSYPK6S325': 'Freedom Mortgage Corp.', 'EQTWLK1G7ODGC2MGLV11': 'Regions Bank', '549300E2UX99HKDBR481': 'Broker Solutions, Inc.', '549300NB3SBC1KHAWB92': 'Gershman Investment Corp.', '549300L8JRY60EOROT34': 'FCB Banks', 'CKVBED0S4DMLKJJ5XH28': 'Stifel Bank & Trust', 'RVDPPPGHCGZ40J4VQ731': 'Pennymac Loan Services LLC', '254900HA4DQWAE0W3342': 'AmeriHome Mortgage Company LLC', 'AD6GFRVSDT01YPT1CS68': 'PNC', '549300CP7747DCRW6U68': 'Carrollton Bank', '549300YDBNK66R6ZES34': 'AB Employees CU', '5493003KZFGGEWCRH554': 'Ditech Financial LLC', '549300LXKO1O7CSK5J52': 'Flat Branch Mortgage, Inc.', '549300DX0B304LAKUN93': 'JLB Corp.', '549300DT7WZ1SOTNFJ62': 'First Bank', '549300O7SGM8FH65GQ47': 'Busey Bank', '5493004T9SQFCNRQ2L76': 'Delmar Financial Co.', 'SS1TRMSN6BRNMOREEV51': 'Flagstar Bank', '549300J7XKT2BI5WX213': 'Caliber Home Loans, Inc.', '549300HW662MN1WU8550': 'United Shore Financial Services LLC', '549300C1ICNCM0V37Y02': 'First State Bank of St. Charles, Missouri', '549300AG64NHILB7ZP05': 'Loan Depot LLC', '254900Q716E7IUMXGB91': 'The Home Loan Expert LLC', '549300DD5QQUHO6PCH70': 'Mortgage Research Center LLC', '549300VSL5DFXBM1II56': 'F&B Acquisition Group LLC', '549300LBCBNR1OT00651': 'Nationstar Mortgage LLC', '549300SP2BPE296CC590': 'The Bank of Edwardsville', '549300BRJZYHYKT4BJ84': 'Home Point Financial Corp.'}

In [4]:
mo_07 = pd.read_csv('./data/public_lar/mo/hmda_2007_mo_all-records_labels.csv', low_memory=False)
mo_08 = pd.read_csv('./data/public_lar/mo/hmda_2008_mo_all-records_labels.csv', low_memory=False)
mo_09 = pd.read_csv('./data/public_lar/mo/hmda_2009_mo_all-records_labels.csv', low_memory=False)
mo_10 = pd.read_csv('./data/public_lar/mo/hmda_2010_mo_all-records_labels.csv', low_memory=False)
mo_11 = pd.read_csv('./data/public_lar/mo/hmda_2011_mo_all-records_labels.csv', low_memory=False)
mo_12 = pd.read_csv('./data/public_lar/mo/hmda_2012_mo_all-records_labels.csv', low_memory=False)
mo_13 = pd.read_csv('./data/public_lar/mo/hmda_2013_mo_all-records_labels.csv', low_memory=False)
mo_14 = pd.read_csv('./data/public_lar/mo/hmda_2014_mo_all-records_labels.csv', low_memory=False)
mo_15 = pd.read_csv('./data/public_lar/mo/hmda_2015_mo_all-records_labels.csv', low_memory=False)
mo_16 = pd.read_csv('./data/public_lar/mo/hmda_2016_mo_all-records_labels.csv', low_memory=False)
mo_17 = pd.read_csv('./data/public_lar/mo/hmda_2017_mo_all-records_labels.csv', low_memory=False)
mo_18 = pd.read_csv('./data/public_lar/mo/missouri.csv', low_memory=False)

In [152]:
mo_18['bank_name'] = mo_18['lei'].map(banks)

In [5]:
il_07 = pd.read_csv('./data/public_lar/il/hmda_2007_il_all-records_labels.csv', low_memory=False)
il_08 = pd.read_csv('./data/public_lar/il/hmda_2008_il_all-records_labels.csv', low_memory=False)
il_09 = pd.read_csv('./data/public_lar/il/hmda_2009_il_all-records_labels.csv', low_memory=False)
il_10 = pd.read_csv('./data/public_lar/il/hmda_2010_il_all-records_labels.csv', low_memory=False)
il_11 = pd.read_csv('./data/public_lar/il/hmda_2011_il_all-records_labels.csv', low_memory=False)
il_12 = pd.read_csv('./data/public_lar/il/hmda_2012_il_all-records_labels.csv', low_memory=False)
il_13 = pd.read_csv('./data/public_lar/il/hmda_2013_il_all-records_labels.csv', low_memory=False)
il_14 = pd.read_csv('./data/public_lar/il/hmda_2014_il_all-records_labels.csv', low_memory=False)
il_15 = pd.read_csv('./data/public_lar/il/hmda_2015_il_all-records_labels.csv', low_memory=False)
il_16 = pd.read_csv('./data/public_lar/il/hmda_2016_il_all-records_labels.csv', low_memory=False)
il_17 = pd.read_csv('./data/public_lar/il/hmda_2017_il_all-records_labels.csv', low_memory=False)
il_18 = pd.read_csv('./data/public_lar/il/illinois.csv', low_memory=False)

In [153]:
il_18['bank_name'] = il_18['lei'].map(banks)

In [154]:
orig07 = mo_07.loc[mo_07.action_taken == 1, 'action_taken'].count()
orig08 = mo_08.loc[mo_08.action_taken == 1, 'action_taken'].count()
orig09 = mo_09.loc[mo_09.action_taken == 1, 'action_taken'].count()
orig10 = mo_10.loc[mo_10.action_taken == 1, 'action_taken'].count()
orig11 = mo_11.loc[mo_11.action_taken == 1, 'action_taken'].count()
orig12 = mo_12.loc[mo_12.action_taken == 1, 'action_taken'].count()
orig13 = mo_13.loc[mo_13.action_taken == 1, 'action_taken'].count()
orig14 = mo_14.loc[mo_14.action_taken == 1, 'action_taken'].count()
orig15 = mo_15.loc[mo_15.action_taken == 1, 'action_taken'].count()
orig16 = mo_16.loc[mo_16.action_taken == 1, 'action_taken'].count()
orig17 = mo_17.loc[mo_17.action_taken == 1, 'action_taken'].count()
orig18 = mo_18.loc[mo_18.action_taken == 1, 'action_taken'].count()

In [155]:
orig_il07 = il_07.loc[il_07.action_taken == 1, 'action_taken'].count()
orig_il08 = il_08.loc[il_08.action_taken == 1, 'action_taken'].count()
orig_il09 = il_09.loc[il_09.action_taken == 1, 'action_taken'].count()
orig_il10 = il_10.loc[il_10.action_taken == 1, 'action_taken'].count()
orig_il11 = il_11.loc[il_11.action_taken == 1, 'action_taken'].count()
orig_il12 = il_12.loc[il_12.action_taken == 1, 'action_taken'].count()
orig_il13 = il_13.loc[il_13.action_taken == 1, 'action_taken'].count()
orig_il14 = il_14.loc[il_14.action_taken == 1, 'action_taken'].count()
orig_il15 = il_15.loc[il_15.action_taken == 1, 'action_taken'].count()
orig_il16 = il_16.loc[il_16.action_taken == 1, 'action_taken'].count()
orig_il17 = il_17.loc[il_17.action_taken == 1, 'action_taken'].count()
orig_il18 = il_18.loc[il_18.action_taken == 1, 'action_taken'].count()

In [156]:
### This is a basic count of records in each year file

m07 = mo_07['as_of_year'].count()
m08 = mo_08['as_of_year'].count()
m09 = mo_09['as_of_year'].count()
m10 = mo_10['as_of_year'].count()
m11 = mo_11['as_of_year'].count()
m12 = mo_12['as_of_year'].count()
m13 = mo_13['as_of_year'].count()
m14 = mo_14['as_of_year'].count()
m15 = mo_15['as_of_year'].count()
m16 = mo_16['as_of_year'].count()
m17 = mo_17['as_of_year'].count()
m18 = mo_18['activity_year'].count()

In [157]:
### This is a basic count of records in each year file

i07 = il_07['as_of_year'].count()
i08 = il_08['as_of_year'].count()
i09 = il_09['as_of_year'].count()
i10 = il_10['as_of_year'].count()
i11 = il_11['as_of_year'].count()
i12 = il_12['as_of_year'].count()
i13 = il_13['as_of_year'].count()
i14 = il_14['as_of_year'].count()
i15 = il_15['as_of_year'].count()
i16 = il_16['as_of_year'].count()
i17 = il_17['as_of_year'].count()
i18 = il_18['activity_year'].count()

In [158]:
count_headers = ['2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018']
mo_orig_counts = [orig07, orig08, orig09, orig10, orig11, orig12, orig13, orig14, orig15, orig16, orig17, orig18]
il_orig_counts = [orig_il07, orig_il08, orig_il09, orig_il10, orig_il11, orig_il12, orig_il13, orig_il14, orig_il15, orig_il16, orig_il17, orig_il18]
mo_orig_counts_df = pd.DataFrame([mo_orig_counts], columns=count_headers)
il_orig_counts_df = pd.DataFrame([il_orig_counts], columns=count_headers)
mo_orig_counts_df['state'] = 'missouri'
il_orig_counts_df['state'] = 'illinois'

In [159]:
final_orig_counts_df = pd.concat([mo_orig_counts_df, il_orig_counts_df])

In [160]:
final_orig_counts = final_orig_counts_df.transpose()

In [161]:
final_orig_counts = final_orig_counts.reset_index()

In [162]:
final_orig_counts.columns = ['year', 'missouri', 'illinois']

In [163]:
fig_orig_mo = px.line(final_orig_counts, x='year', y='missouri')

In [164]:
fig_orig_mo.update_yaxes(rangemode="tozero")

In [165]:
fig_orig_il = px.line(final_orig_counts, x='year', y='illinois')

In [166]:
fig_orig_il.update_yaxes(rangemode="tozero")

In [167]:
count_headers = ['2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018']
mo_counts = [m07, m08, m09, m10, m11, m12, m13, m14, m15, m16, m17, m18]
il_counts = [i07, i08, i09, i10, i11, i12, i13, i14, i15, i16, i17, i18]
mo_counts_df = pd.DataFrame([mo_counts], columns=count_headers)
il_counts_df = pd.DataFrame([il_counts], columns=count_headers)
mo_counts_df['state'] = 'missouri'
il_counts_df['state'] = 'illinois'

In [168]:
final_counts_df = pd.concat([mo_counts_df, il_counts_df])

In [169]:
final_counts = final_counts_df.transpose()

In [170]:
final_counts = final_counts.reset_index()

In [171]:
final_counts.columns = ['year', 'missouri', 'illinois']

In [172]:
fig_mo = px.line(final_counts, x='year', y='missouri')

In [173]:
fig_mo.update_yaxes(rangemode="tozero")

In [174]:
fig_il = px.line(final_counts, x='year', y='illinois')
fig_il.update_yaxes(rangemode="tozero")

In [175]:
### Filtering to just the St. Louis metro in Missouri

stl_07 = mo_07.loc[mo_07['msamd'].isin(stl)]
stl_08 = mo_08.loc[mo_08['msamd'].isin(stl)]
stl_09 = mo_09.loc[mo_09['msamd'].isin(stl)]
stl_10 = mo_10.loc[mo_10['msamd'].isin(stl)]
stl_11 = mo_11.loc[mo_11['msamd'].isin(stl)]
stl_12 = mo_12.loc[mo_12['msamd'].isin(stl)]
stl_13 = mo_13.loc[mo_13['msamd'].isin(stl)]
stl_14 = mo_14.loc[mo_14['msamd'].isin(stl)]
stl_15 = mo_15.loc[mo_15['msamd'].isin(stl)]
stl_16 = mo_16.loc[mo_16['msamd'].isin(stl)]
stl_17 = mo_17.loc[mo_17['msamd'].isin(stl)]
stl_18 = mo_18.loc[mo_18['derived_msa_md'].isin(stl)]

In [176]:
### Filtering to just the St. Louis metro in Illinois

stl_07_il = il_07.loc[il_07['msamd'].isin(stl)]
stl_08_il = il_08.loc[il_08['msamd'].isin(stl)]
stl_09_il = il_09.loc[il_09['msamd'].isin(stl)]
stl_10_il = il_10.loc[il_10['msamd'].isin(stl)]
stl_11_il = il_11.loc[il_11['msamd'].isin(stl)]
stl_12_il = il_12.loc[il_12['msamd'].isin(stl)]
stl_13_il = il_13.loc[il_13['msamd'].isin(stl)]
stl_14_il = il_14.loc[il_14['msamd'].isin(stl)]
stl_15_il = il_15.loc[il_15['msamd'].isin(stl)]
stl_16_il = il_16.loc[il_16['msamd'].isin(stl)]
stl_17_il = il_17.loc[il_17['msamd'].isin(stl)]
stl_18_il = il_18.loc[il_18['derived_msa_md'].isin(stl)]

In [177]:
### Concatenating the metro dfs from each separate state, of the same year, into one df, so there is one 
### df for each year in the metro

metro07 = pd.concat([stl_07, stl_07_il])
metro08 = pd.concat([stl_08, stl_08_il])
metro09 = pd.concat([stl_09, stl_09_il])
metro10 = pd.concat([stl_10, stl_10_il])
metro11 = pd.concat([stl_11, stl_11_il])
metro12 = pd.concat([stl_12, stl_12_il])
metro13 = pd.concat([stl_13, stl_13_il])
metro14 = pd.concat([stl_14, stl_14_il])
metro15 = pd.concat([stl_15, stl_15_il])
metro16 = pd.concat([stl_16, stl_16_il])
metro17 = pd.concat([stl_17, stl_17_il])
metro18 = pd.concat([stl_18, stl_18_il])

In [178]:
metro_files = [metro07, metro08, metro09, metro10, metro11, metro12, metro13, metro14, metro15, metro16, metro17]
metro0717 = pd.concat(metro_files)

In [179]:
# Filtering the metro area dfs to just originated loans

metro_orig07 = metro07.loc[metro07.action_taken == 1, 'action_taken'].count()
metro_orig08 = metro08.loc[metro08.action_taken == 1, 'action_taken'].count()
metro_orig09 = metro09.loc[metro09.action_taken == 1, 'action_taken'].count()
metro_orig10 = metro10.loc[metro10.action_taken == 1, 'action_taken'].count()
metro_orig11 = metro11.loc[metro11.action_taken == 1, 'action_taken'].count()
metro_orig12 = metro12.loc[metro12.action_taken == 1, 'action_taken'].count()
metro_orig13 = metro13.loc[metro13.action_taken == 1, 'action_taken'].count()
metro_orig14 = metro14.loc[metro14.action_taken == 1, 'action_taken'].count()
metro_orig15 = metro15.loc[metro15.action_taken == 1, 'action_taken'].count()
metro_orig16 = metro16.loc[metro16.action_taken == 1, 'action_taken'].count()
metro_orig17 = metro17.loc[metro17.action_taken == 1, 'action_taken'].count()
metro_orig18 = metro18.loc[metro18.action_taken == 1, 'action_taken'].count()

In [180]:
# Counting the number of loan applications in the metro area, for context/reference

metro07c = metro07['as_of_year'].count()
metro08c = metro08['as_of_year'].count()
metro09c = metro09['as_of_year'].count()
metro10c = metro10['as_of_year'].count()
metro11c = metro11['as_of_year'].count()
metro12c = metro12['as_of_year'].count()
metro13c = metro13['as_of_year'].count()
metro14c = metro14['as_of_year'].count()
metro15c = metro15['as_of_year'].count()
metro16c = metro16['as_of_year'].count()
metro17c = metro17['as_of_year'].count()
metro18c = metro18['activity_year'].count()

print metro17c

143816


In [181]:
metro_orig_counts = [metro_orig07, metro_orig08, metro_orig09, metro_orig10, metro_orig11, metro_orig12, metro_orig13, metro_orig14, metro_orig15, metro_orig16, metro_orig17, metro_orig18]
metro_orig_counts_df = pd.DataFrame([metro_orig_counts], columns=count_headers)

In [182]:
metro_orig_counts_df = metro_orig_counts_df.transpose()

In [183]:
metro_orig_counts_df = metro_orig_counts_df.reset_index()

In [184]:
metro_orig_counts_df.columns = ['year', 'metro']

In [185]:
fig_metro_orig = px.line(metro_orig_counts_df, x='year', y='metro')
fig_metro_orig.update_yaxes(rangemode="tozero")

In [186]:
# Filtering the metro year dataframes to only originated loans

metro_orig07 = metro07.loc[metro07['action_taken'].isin(orig)]
metro_orig08 = metro08.loc[metro08['action_taken'].isin(orig)]
metro_orig09 = metro09.loc[metro09['action_taken'].isin(orig)]
metro_orig10 = metro10.loc[metro10['action_taken'].isin(orig)]
metro_orig11 = metro11.loc[metro11['action_taken'].isin(orig)]
metro_orig12 = metro12.loc[metro12['action_taken'].isin(orig)]
metro_orig13 = metro13.loc[metro13['action_taken'].isin(orig)]
metro_orig14 = metro14.loc[metro14['action_taken'].isin(orig)]
metro_orig15 = metro15.loc[metro15['action_taken'].isin(orig)]
metro_orig16 = metro16.loc[metro16['action_taken'].isin(orig)]
metro_orig17 = metro17.loc[metro17['action_taken'].isin(orig)]
metro_orig18 = metro18.loc[metro18['action_taken'].isin(orig)]

In [187]:
# Filtering the dfs of originated loans to only sf

metsf07 = metro_orig07.loc[metro_orig07['property_type'].isin(sf_old)]
metsf08 = metro_orig08.loc[metro_orig08['property_type'].isin(sf_old)]
metsf09 = metro_orig09.loc[metro_orig09['property_type'].isin(sf_old)]
metsf10 = metro_orig10.loc[metro_orig10['property_type'].isin(sf_old)]
metsf11 = metro_orig11.loc[metro_orig11['property_type'].isin(sf_old)]
metsf12 = metro_orig12.loc[metro_orig12['property_type'].isin(sf_old)]
metsf13 = metro_orig13.loc[metro_orig13['property_type'].isin(sf_old)]
metsf14 = metro_orig14.loc[metro_orig14['property_type'].isin(sf_old)]
metsf15 = metro_orig15.loc[metro_orig15['property_type'].isin(sf_old)]
metsf16 = metro_orig16.loc[metro_orig16['property_type'].isin(sf_old)]
metsf17 = metro_orig17.loc[metro_orig17['property_type'].isin(sf_old)]
metsf18 = metro_orig18.loc[metro_orig18['derived_dwelling_category'].isin(sf)]

metsf07c = metsf07['as_of_year'].count()
metsf08c = metsf08['as_of_year'].count()
metsf09c = metsf09['as_of_year'].count()
metsf10c = metsf10['as_of_year'].count()
metsf11c = metsf11['as_of_year'].count()
metsf12c = metsf12['as_of_year'].count()
metsf13c = metsf13['as_of_year'].count()
metsf14c = metsf14['as_of_year'].count()
metsf15c = metsf15['as_of_year'].count()
metsf16c = metsf16['as_of_year'].count()
metsf17c = metsf17['as_of_year'].count()
metsf18c = metsf18['activity_year'].count()

metro_sf_counts = [metsf07c, metsf08c, metsf09c, metsf10c, metsf11c, metsf12c, metsf13c, metsf14c, metsf15c, metsf16c, metsf17c, metsf18c]
metro_sf_counts_df = pd.DataFrame([metro_sf_counts], columns=count_headers)

In [188]:
metro_sf_counts_df = metro_sf_counts_df.transpose()
metro_sf_counts_df = metro_sf_counts_df.reset_index()

In [189]:
metro_sf_counts_df.columns = ['year', 'metro orig sf']
fig_metro_orig_sf = px.line(metro_sf_counts_df, x='year', y='metro orig sf')
fig_metro_orig_sf.update_yaxes(rangemode="tozero")

In [190]:
# Filtering the dfs of originated, sf properties loans to originated, sf properties for home purchase

metsfp07 = metsf07.loc[metsf07['loan_purpose'].isin(purchase)]
metsfp08 = metsf08.loc[metsf08['loan_purpose'].isin(purchase)]
metsfp09 = metsf09.loc[metsf09['loan_purpose'].isin(purchase)]
metsfp10 = metsf10.loc[metsf10['loan_purpose'].isin(purchase)]
metsfp11 = metsf11.loc[metsf11['loan_purpose'].isin(purchase)]
metsfp12 = metsf12.loc[metsf12['loan_purpose'].isin(purchase)]
metsfp13 = metsf13.loc[metsf13['loan_purpose'].isin(purchase)]
metsfp14 = metsf14.loc[metsf14['loan_purpose'].isin(purchase)]
metsfp15 = metsf15.loc[metsf15['loan_purpose'].isin(purchase)]
metsfp16 = metsf16.loc[metsf16['loan_purpose'].isin(purchase)]
metsfp17 = metsf17.loc[metsf17['loan_purpose'].isin(purchase)]
metsfp18 = metsf18.loc[metsf18['loan_purpose'].isin(purchase)]

metsfp07c = metsfp07['as_of_year'].count()
metsfp08c = metsfp08['as_of_year'].count()
metsfp09c = metsfp09['as_of_year'].count()
metsfp10c = metsfp10['as_of_year'].count()
metsfp11c = metsfp11['as_of_year'].count()
metsfp12c = metsfp12['as_of_year'].count()
metsfp13c = metsfp13['as_of_year'].count()
metsfp14c = metsfp14['as_of_year'].count()
metsfp15c = metsfp15['as_of_year'].count()
metsfp16c = metsfp16['as_of_year'].count()
metsfp17c = metsfp17['as_of_year'].count()
metsfp18c = metsfp18['activity_year'].count()

metro_sf_purchases = [metsfp07c, metsfp08c, metsfp09c, metsfp10c, metsfp11c, metsfp12c, metsfp13c, metsfp14c, metsfp15c, metsfp16c, metsfp17c, metsfp18c]
metro_sf_purchases_df = pd.DataFrame([metro_sf_purchases], columns=count_headers)

In [191]:
metro_sf_purchases_df = metro_sf_purchases_df.transpose()
metro_sf_purchases_df = metro_sf_purchases_df.reset_index()

In [192]:
metro_sf_purchases_df.columns = ['year', 'metro orig sf purchases']
fig_metro_orig_sf_p = px.line(metro_sf_purchases_df, x='year', y='metro orig sf purchases')
fig_metro_orig_sf_p.update_yaxes(rangemode="tozero")

In [193]:
# Filtering the dfs of originated loans to home purchase (of all property types, not just sf)

methp07 = metro_orig07.loc[metro_orig07['loan_purpose'].isin(purchase)]
methp08 = metro_orig08.loc[metro_orig08['loan_purpose'].isin(purchase)]
methp09 = metro_orig09.loc[metro_orig09['loan_purpose'].isin(purchase)]
methp10 = metro_orig10.loc[metro_orig10['loan_purpose'].isin(purchase)]
methp11 = metro_orig11.loc[metro_orig11['loan_purpose'].isin(purchase)]
methp12 = metro_orig12.loc[metro_orig12['loan_purpose'].isin(purchase)]
methp13 = metro_orig13.loc[metro_orig13['loan_purpose'].isin(purchase)]
methp14 = metro_orig14.loc[metro_orig14['loan_purpose'].isin(purchase)]
methp15 = metro_orig15.loc[metro_orig15['loan_purpose'].isin(purchase)]
methp16 = metro_orig16.loc[metro_orig16['loan_purpose'].isin(purchase)]
methp17 = metro_orig17.loc[metro_orig17['loan_purpose'].isin(purchase)]
methp18 = metro_orig18.loc[metro_orig18['loan_purpose'].isin(purchase)]

methp07c = methp07['as_of_year'].count()
methp08c = methp08['as_of_year'].count()
methp09c = methp09['as_of_year'].count()
methp10c = methp10['as_of_year'].count()
methp11c = methp11['as_of_year'].count()
methp12c = methp12['as_of_year'].count()
methp13c = methp13['as_of_year'].count()
methp14c = methp14['as_of_year'].count()
methp15c = methp15['as_of_year'].count()
methp16c = methp16['as_of_year'].count()
methp17c = methp17['as_of_year'].count()
methp18c = methp18['activity_year'].count()

metro_purchases = [methp07c, methp08c, methp09c, methp10c, methp11c, methp12c, methp13c, methp14c, methp15c, methp16c, methp17c, methp18c]
metro_purchases_df = pd.DataFrame([metro_purchases], columns=count_headers)

In [194]:
metro_purchases_df = metro_purchases_df.transpose()
metro_purchases_df = metro_purchases_df.reset_index()

In [195]:
metro_purchases_df.columns = ['year', 'metro orig purchases']
fig_metro_orig_p = px.line(metro_purchases_df, x='year', y='metro orig purchases')
fig_metro_orig_p.update_yaxes(rangemode="tozero")

In [196]:
# Filtering the dfs of orignated loans to refinance loans (of all property types, not just sf)

metr07 = metro_orig07.loc[metro_orig07['loan_purpose'].isin(refi_old)]
metr08 = metro_orig08.loc[metro_orig08['loan_purpose'].isin(refi_old)]
metr09 = metro_orig09.loc[metro_orig09['loan_purpose'].isin(refi_old)]
metr10 = metro_orig10.loc[metro_orig10['loan_purpose'].isin(refi_old)]
metr11 = metro_orig11.loc[metro_orig11['loan_purpose'].isin(refi_old)]
metr12 = metro_orig12.loc[metro_orig12['loan_purpose'].isin(refi_old)]
metr13 = metro_orig13.loc[metro_orig13['loan_purpose'].isin(refi_old)]
metr14 = metro_orig14.loc[metro_orig14['loan_purpose'].isin(refi_old)]
metr15 = metro_orig15.loc[metro_orig15['loan_purpose'].isin(refi_old)]
metr16 = metro_orig16.loc[metro_orig16['loan_purpose'].isin(refi_old)]
metr17 = metro_orig17.loc[metro_orig17['loan_purpose'].isin(refi_old)]
metr18 = metro_orig18.loc[metro_orig18['loan_purpose'].isin(refi)]

metr07c = metr07['as_of_year'].count()
metr08c = metr08['as_of_year'].count()
metr09c = metr09['as_of_year'].count()
metr10c = metr10['as_of_year'].count()
metr11c = metr11['as_of_year'].count()
metr12c = metr12['as_of_year'].count()
metr13c = metr13['as_of_year'].count()
metr14c = metr14['as_of_year'].count()
metr15c = metr15['as_of_year'].count()
metr16c = metr16['as_of_year'].count()
metr17c = metr17['as_of_year'].count()
metr18c = metr18['activity_year'].count()

metro_refis = [metr07c, metr08c, metr09c, metr10c, metr11c, metr12c, metr13c, metr14c, metr15c, metr16c, metr17c, metr18c]
metro_refis_df = pd.DataFrame([metro_refis], columns=count_headers)

In [197]:
metro_refis_df = metro_refis_df.transpose()
metro_refis_df = metro_refis_df.reset_index()

In [198]:
metro_refis_df.columns = ['year', 'metro orig refis']
fig_metro_orig_p = px.line(metro_refis_df, x='year', y='metro orig refis')
fig_metro_orig_p.update_yaxes(rangemode="tozero")

In [199]:
stl_race_groups_07 = metro07.groupby(['applicant_race_name_1']).size()
stl_race_groups_08 = metro08.groupby(['applicant_race_name_1']).size()
stl_race_groups_09 = metro09.groupby(['applicant_race_name_1']).size()
stl_race_groups_10 = metro10.groupby(['applicant_race_name_1']).size()
stl_race_groups_11 = metro11.groupby(['applicant_race_name_1']).size()
stl_race_groups_12 = metro12.groupby(['applicant_race_name_1']).size()
stl_race_groups_13 = metro13.groupby(['applicant_race_name_1']).size()
stl_race_groups_14 = metro14.groupby(['applicant_race_name_1']).size()
stl_race_groups_15 = metro15.groupby(['applicant_race_name_1']).size()
stl_race_groups_16 = metro16.groupby(['applicant_race_name_1']).size()
stl_race_groups_17 = metro17.groupby(['applicant_race_name_1']).size()
stl_race_groups_18 = metro18.groupby(['applicant_race_1']).size()

In [200]:
stl_race_groups = [stl_race_groups_07, stl_race_groups_08, stl_race_groups_09, stl_race_groups_10, stl_race_groups_11, stl_race_groups_12, stl_race_groups_13, stl_race_groups_14, stl_race_groups_15, stl_race_groups_16, stl_race_groups_17, stl_race_groups_18]

In [201]:
stl_black07 = stl_race_groups_07['Black or African American']
stl_black08 = stl_race_groups_08['Black or African American']
stl_black09 = stl_race_groups_09['Black or African American']
stl_black10 = stl_race_groups_10['Black or African American']
stl_black11 = stl_race_groups_11['Black or African American']
stl_black12 = stl_race_groups_12['Black or African American']
stl_black13 = stl_race_groups_13['Black or African American']
stl_black14 = stl_race_groups_14['Black or African American']
stl_black15 = stl_race_groups_15['Black or African American']
stl_black16 = stl_race_groups_16['Black or African American']
stl_black17 = stl_race_groups_17['Black or African American']
stl_black18 = stl_race_groups_18[3]
stl_blacks = [stl_black07, stl_black08, stl_black09, stl_black10, stl_black11, stl_black12, stl_black13, stl_black14, stl_black15, stl_black16, stl_black17, stl_black18]

In [202]:
print stl_blacks

[39097, 22525, 14753, 10552, 8242, 10223, 11084, 8500, 9509, 10990, 10236, 9857]


In [203]:
stl_white07 = stl_race_groups_07['White']
stl_white08 = stl_race_groups_08['White']
stl_white09 = stl_race_groups_09['White']
stl_white10 = stl_race_groups_10['White']
stl_white11 = stl_race_groups_11['White']
stl_white12 = stl_race_groups_12['White']
stl_white13 = stl_race_groups_13['White']
stl_white14 = stl_race_groups_14['White']
stl_white15 = stl_race_groups_15['White']
stl_white16 = stl_race_groups_16['White']
stl_white17 = stl_race_groups_17['White']
stl_white18 = stl_race_groups_18[5]
stl_whites = [stl_white07, stl_white08, stl_white09, stl_white10, stl_white11, stl_white12, stl_white13, stl_white14, stl_white15, stl_white16, stl_white17, stl_white18]

In [204]:
print stl_whites

[182589, 139917, 181307, 159137, 125928, 164360, 137485, 82860, 101741, 115192, 98313, 94098]


In [205]:
### What percentage of applications in the STL area came from black people?

blackpct07 = (stl_black07*100)/metro07c
blackpct08 = (stl_black08*100)/metro08c
blackpct09 = (stl_black09*100)/metro09c
blackpct10 = (stl_black10*100)/metro10c
blackpct11 = (stl_black11*100)/metro11c
blackpct12 = (stl_black12*100)/metro12c
blackpct13 = (stl_black13*100)/metro13c
blackpct14 = (stl_black14*100)/metro14c
blackpct15 = (stl_black15*100)/metro15c
blackpct16 = (stl_black16*100)/metro16c
blackpct17 = (stl_black17*100)/metro17c
blackpct18 = (stl_black18*100)/metro18c
blackpcts = [blackpct07, blackpct08, blackpct09, blackpct10, blackpct11, blackpct12, blackpct13, blackpct14, blackpct15, blackpct16, blackpct17, blackpct18]
print blackpcts

[13, 10, 5, 4, 4, 4, 5, 6, 6, 6, 7, 7]


In [206]:
### What percentage of applications in the STL area came from white people?

whitepct07 = (stl_white07*100)/metro07c
whitepct08 = (stl_white08*100)/metro08c
whitepct09 = (stl_white09*100)/metro09c
whitepct10 = (stl_white10*100)/metro10c
whitepct11 = (stl_white11*100)/metro11c
whitepct12 = (stl_white12*100)/metro12c
whitepct13 = (stl_white13*100)/metro13c
whitepct14 = (stl_white14*100)/metro14c
whitepct15 = (stl_white15*100)/metro15c
whitepct16 = (stl_white16*100)/metro16c
whitepct17 = (stl_white17*100)/metro17c
whitepct18 = (stl_white18*100)/metro18c
whitepcts = [whitepct07, whitepct08, whitepct09, whitepct10, whitepct11, whitepct12, whitepct13, whitepct14, whitepct15, whitepct16, whitepct17, whitepct18]
print whitepcts

[62, 66, 69, 74, 72, 74, 71, 66, 67, 68, 68, 68]


In [207]:
### What percentage of loans originated in the STL area went to black people?

# This filters the STL metro area originated loans to only originated loans made to black people

metb07 = metro_orig07.loc[metro_orig07['applicant_race_name_1'].isin(black_old)]
metb08 = metro_orig08.loc[metro_orig08['applicant_race_name_1'].isin(black_old)]
metb09 = metro_orig09.loc[metro_orig09['applicant_race_name_1'].isin(black_old)]
metb10 = metro_orig10.loc[metro_orig10['applicant_race_name_1'].isin(black_old)]
metb11 = metro_orig11.loc[metro_orig11['applicant_race_name_1'].isin(black_old)]
metb12 = metro_orig12.loc[metro_orig12['applicant_race_name_1'].isin(black_old)]
metb13 = metro_orig13.loc[metro_orig13['applicant_race_name_1'].isin(black_old)]
metb14 = metro_orig14.loc[metro_orig14['applicant_race_name_1'].isin(black_old)]
metb15 = metro_orig15.loc[metro_orig15['applicant_race_name_1'].isin(black_old)]
metb16 = metro_orig16.loc[metro_orig16['applicant_race_name_1'].isin(black_old)]
metb17 = metro_orig17.loc[metro_orig17['applicant_race_name_1'].isin(black_old)]
metb18 = metro_orig18.loc[metro_orig18['applicant_race_1'].isin(black_new)]

# This creates variables containing a count of number of loans made to black people every year

metb07c = metb07['as_of_year'].count()
metb08c = metb08['as_of_year'].count()
metb09c = metb09['as_of_year'].count()
metb10c = metb10['as_of_year'].count()
metb11c = metb11['as_of_year'].count()
metb12c = metb12['as_of_year'].count()
metb13c = metb13['as_of_year'].count()
metb14c = metb14['as_of_year'].count()
metb15c = metb15['as_of_year'].count()
metb16c = metb16['as_of_year'].count()
metb17c = metb17['as_of_year'].count()
metb18c = metb18['activity_year'].count()
metbcounts = [metb07c, metb08c, metb09c, metb10c, metb11c, metb12c, metb13c, metb14c, metb15c, metb16c, metb17c, metb18c]

In [208]:
print metbcounts

[12553, 6803, 5814, 4189, 3432, 4719, 4790, 3678, 4209, 4877, 4652, 4437]


In [209]:
### What percentage of loans originated in the STL area went to white people?

# This filters the STL metro area originated loans to only originated loans made to white people

metw07 = metro_orig07.loc[metro_orig07['applicant_race_name_1'].isin(white_old)]
metw08 = metro_orig08.loc[metro_orig08['applicant_race_name_1'].isin(white_old)]
metw09 = metro_orig09.loc[metro_orig09['applicant_race_name_1'].isin(white_old)]
metw10 = metro_orig10.loc[metro_orig10['applicant_race_name_1'].isin(white_old)]
metw11 = metro_orig11.loc[metro_orig11['applicant_race_name_1'].isin(white_old)]
metw12 = metro_orig12.loc[metro_orig12['applicant_race_name_1'].isin(white_old)]
metw13 = metro_orig13.loc[metro_orig13['applicant_race_name_1'].isin(white_old)]
metw14 = metro_orig14.loc[metro_orig14['applicant_race_name_1'].isin(white_old)]
metw15 = metro_orig15.loc[metro_orig15['applicant_race_name_1'].isin(white_old)]
metw16 = metro_orig16.loc[metro_orig16['applicant_race_name_1'].isin(white_old)]
metw17 = metro_orig17.loc[metro_orig17['applicant_race_name_1'].isin(white_old)]
metw18 = metro_orig18.loc[metro_orig18['applicant_race_1'].isin(white_new)]

# This creates variables containing a count of number of loans made to white people every year

metw07c = metw07['as_of_year'].count()
metw08c = metw08['as_of_year'].count()
metw09c = metw09['as_of_year'].count()
metw10c = metw10['as_of_year'].count()
metw11c = metw11['as_of_year'].count()
metw12c = metw12['as_of_year'].count()
metw13c = metw13['as_of_year'].count()
metw14c = metw14['as_of_year'].count()
metw15c = metw15['as_of_year'].count()
metw16c = metw16['as_of_year'].count()
metw17c = metw17['as_of_year'].count()
metw18c = metw18['activity_year'].count()
metwcounts = [metw07c, metw08c, metw09c, metw10c, metw11c, metw12c, metw13c, metw14c, metw15c, metw16c, metw17c, metw18c]

In [210]:
print metwcounts

[91761, 74552, 105822, 94731, 75601, 101998, 81264, 51462, 65762, 73082, 59667, 56201]


In [221]:
### Which banks had the most applications for loans in the STL metro area in 2018? 
stl_banks_apps18 = metro18.groupby(['lei']).size()
stl_banks_apps18 = stl_banks_apps18.reset_index()
stl_banks_apps18.columns = ['respondent_id', 'count']
stl_banks_apps18['bank_name'] = stl_banks_apps18['respondent_id'].map(banks)
stl_banks_apps18 = stl_banks_apps18.sort_values('count', ascending=False)
print stl_banks_apps18

            respondent_id  count                                  bank_name
630  6BYL5QZYBDK8S7L73M02   8914                                    US Bank
660  KB1H1DSPRFMYMCUFXT09   8392                                Wells Fargo
638  B4TYDEB6GKMZO031MB27   4916                                        BOA
634  7H6GLXDRUGQFU57RNE97   3769                            JP Morgan/Chase
531  549300T67186NPGCHI50   3693                            First Community
318  549300FGXN1K3HLB1R50   3572                                    Quicken
233  549300AQ3T62GXDU7D76   3341                     Guild Mortgage Company
627  549300ZZME37MXI1EF14   3158                            DAS Acquisition
632  7DMUJTL9FFTVIAG9H788   3066                              Commerce Bank
410  549300LYRWPSYPK6S325   2581                     Freedom Mortgage Corp.
648  EQTWLK1G7ODGC2MGLV11   2388                               Regions Bank
304  549300E2UX99HKDBR481   2307                     Broker Solutions, Inc.
433  549300N

In [220]:
### Which banks originated the most loans in the STL metro area in 2018? 

stl_banks18 = metro_orig18.groupby(['lei']).size()
stl_banks18 = stl_banks18.reset_index()
stl_banks18.columns = ['respondent_id', 'count']
stl_banks18['bank_name'] = stl_banks18['respondent_id'].map(banks)
stl_banks18 = stl_banks18.sort_values('count', ascending=False)
print stl_banks18

            respondent_id  count                                  bank_name
562  6BYL5QZYBDK8S7L73M02   4187                                    US Bank
559  549300ZZME37MXI1EF14   2685                            DAS Acquisition
206  549300AQ3T62GXDU7D76   2547                     Guild Mortgage Company
283  549300FGXN1K3HLB1R50   2308                                    Quicken
570  B4TYDEB6GKMZO031MB27   2077                                        BOA
469  549300T67186NPGCHI50   1943                            First Community
573  CKVBED0S4DMLKJJ5XH28   1911                        Stifel Bank & Trust
564  7DMUJTL9FFTVIAG9H788   1801                              Commerce Bank
352  549300L8JRY60EOROT34   1712                                  FCB Banks
384  549300NB3SBC1KHAWB92   1693                  Gershman Investment Corp.
271  549300E2UX99HKDBR481   1641                     Broker Solutions, Inc.
236  549300CP7747DCRW6U68   1400                            Carrollton Bank
363  549300L

In [55]:
#############################

In [118]:
final_mo_groups_race = final_mo_stl.groupby(['applicant_race_name_1', 'as_of_year']).size()
print final_mo_groups_race['Black or African American']

as_of_year
2007    33189
2008    19031
2009    12137
2010     8560
2011     6517
2012     8110
2013     8943
2014     6953
2015     7803
2016     8806
2017     8230
dtype: int64


In [119]:
print final_mo_groups_race['White']

as_of_year
2007    142000
2008    107204
2009    140415
2010    123035
2011     96260
2012    126827
2013    107269
2014     63407
2015     79743
2016     90646
2017     76956
dtype: int64


In [126]:
stl_18_groups_race = stl_18.groupby(['applicant_race_1', 'activity_year']).size()
print stl_18_groups_race[3]

activity_year
2018    7990
dtype: int64


In [127]:
print stl_18_groups_race[5]

activity_year
2018    74723
dtype: int64


In [114]:
stl_18_il_groups = stl_18_il.groupby(['derived_dwelling_category', 'activity_year']).size()
print stl_18_il_groups

derived_dwelling_category               activity_year
Multifamily:Manufactured                2018                 6
Multifamily:Site-Built                  2018               183
Single Family (1-4 Units):Manufactured  2018               538
Single Family (1-4 Units):Site-Built    2018             26392
dtype: int64


In [113]:
final_il_groups = final_il_stl.groupby(['property_type', 'as_of_year']).size()
print final_il_groups

property_type  as_of_year
1              2007          57600
               2008          42730
               2009          52903
               2010          44953
               2011          37486
               2012          45483
               2013          38213
               2014          26089
               2015          29477
               2016          32501
               2017          28535
2              2007            997
               2008            776
               2009            577
               2010            770
               2011            661
               2012            617
               2013            536
               2014            570
               2015            561
               2016            519
               2017            565
3              2007            120
               2008            173
               2009             98
               2010             99
               2011             91
               2012          

In [112]:
stl_18_groups = stl_18.groupby(['derived_dwelling_category', 'activity_year']).size()
print stl_18_groups

derived_dwelling_category               activity_year
Multifamily:Manufactured                2018                 18
Multifamily:Site-Built                  2018                467
Single Family (1-4 Units):Manufactured  2018               1328
Single Family (1-4 Units):Site-Built    2018             107944
dtype: int64


In [109]:
final_mo_groups = final_mo_stl.groupby(['property_type', 'as_of_year']).size()
print final_mo_groups

property_type  as_of_year
1              2007          229141
               2008          164832
               2009          206116
               2010          164975
               2011          134833
               2012          172609
               2013          150035
               2014           95610
               2015          118988
               2016          132800
               2017          112693
2              2007            2975
               2008            2297
               2009            1478
               2010            1391
               2011            1435
               2012            1598
               2013            1898
               2014            1157
               2015            1080
               2016            1281
               2017            1538
3              2007             391
               2008             409
               2009             232
               2010             248
               2011             262
  

In [23]:
stl_07_c = stl_07['as_of_year'].count()
stl_08_c = stl_08['as_of_year'].count()
stl_09_c = stl_09['as_of_year'].count()
stl_10_c = stl_10['as_of_year'].count()
stl_11_c = stl_11['as_of_year'].count()
stl_12_c = stl_12['as_of_year'].count()
stl_13_c = stl_13['as_of_year'].count()
stl_14_c = stl_14['as_of_year'].count()
stl_15_c = stl_15['as_of_year'].count()
stl_16_c = stl_16['as_of_year'].count()
stl_17_c = stl_17['as_of_year'].count()
stl_18_c = stl_18['activity_year'].count()

### Illinois

stl_07_il_c = stl_07_il['as_of_year'].count()
stl_08_il_c = stl_08_il['as_of_year'].count()
stl_09_il_c = stl_09_il['as_of_year'].count()
stl_10_il_c = stl_10_il['as_of_year'].count()
stl_11_il_c = stl_11_il['as_of_year'].count()
stl_12_il_c = stl_12_il['as_of_year'].count()
stl_13_il_c = stl_13_il['as_of_year'].count()
stl_14_il_c = stl_14_il['as_of_year'].count()
stl_15_il_c = stl_15_il['as_of_year'].count()
stl_16_il_c = stl_16_il['as_of_year'].count()
stl_17_il_c = stl_17_il['as_of_year'].count()
stl_18_il_c = stl_18_il['activity_year'].count()

stl_counts_mo = [stl_07_c, stl_08_c, stl_09_c, stl_10_c, stl_11_c, stl_12_c, stl_13_c, stl_14_c, stl_15_c, stl_16_c, stl_17_c, stl_18_c]
stl_counts_il = [stl_07_il_c, stl_08_il_c, stl_09_il_c, stl_10_il_c, stl_11_il_c, stl_12_il_c, stl_13_il_c, stl_14_il_c, stl_15_il_c, stl_16_il_c, stl_17_il_c, stl_18_il_c]

stl_mo_df = pd.DataFrame([stl_counts_mo], columns=count_headers)
stl_il_df = pd.DataFrame([stl_counts_il], columns=count_headers)

stl_mo_df['state'] = 'missouri'
stl_il_df['state'] = 'illinois'

stl_final_df = pd.concat([stl_mo_df, stl_il_df])
stl_final = stl_final_df.transpose()
stl_final = stl_final.reset_index()
stl_final.columns = ['year', 'missouri', 'illinois']
stl_final = stl_final.drop(stl_final.index[12])
stl_final['both'] = stl_final['missouri'] + stl_final['illinois']
stl_fig = px.line(stl_final, x='year', y='both')
stl_fig.update_yaxes(rangemode="tozero")

In [74]:
stl_mo_all_years_list = [stl_07, stl_08, stl_09, stl_10, stl_11, stl_12, stl_13, stl_14, stl_15, stl_16, stl_17]
stl_il_all_years_list = [stl_07_il, stl_08_il, stl_09_il, stl_10_il, stl_11_il, stl_12_il, stl_13_il, stl_14_il, stl_15_il, stl_16_il, stl_17_il]
final_mo_stl = pd.concat(stl_mo_all_years_list)
final_il_stl = pd.concat(stl_il_all_years_list)
final_mo_stl.to_csv('stl_mo_07-17.csv')
stl_18.to_csv('stl_mo_18.csv')
final_il_stl.to_csv('stl_il_07-17.csv')
stl_18_il.to_csv('stl_il_18.csv')

In [120]:
### Putting the two dataframes into a list

stl_frames_list = [stl_frame1, stl_frame2]

In [121]:
### Concatenating the two separate StL dfs into one, exporting a .csv of loan applications in the StL metro area

stl_frames = pd.concat(stl_frames_list)
stl_frames.to_csv('./data/public_lar/stl_metro.csv')

In [151]:
### Df for StL city

stl_city_df = stl_frames.loc[stl_frames['county_code'].isin(stl_city)]

In [152]:
### Checking out StL city info

stl_city_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11507 entries, 46 to 162491
Data columns (total 100 columns):
Unnamed: 0                                  11507 non-null int64
activity_year                               11507 non-null int64
lei                                         11507 non-null object
derived_msa_md                              11507 non-null int64
state_code                                  11507 non-null object
county_code                                 11507 non-null object
census_tract                                11507 non-null object
conforming_loan_limit                       11306 non-null object
derived_loan_product_type                   11507 non-null object
derived_dwelling_category                   11507 non-null object
derived_ethnicity                           11507 non-null object
derived_race                                11507 non-null object
derived_sex                                 11507 non-null object
action_taken                     

In [153]:
### Filtering to just single-family homes in the metro

stl_sf = stl_frames.loc[stl_frames['derived_dwelling_category'].isin(sf)]

### To just single-family in the city

stl_city_sf = stl_sf.loc[stl_sf['county_code'].isin(stl_city)]

In [161]:
### Checking out how many applications for single-family mortgages

stl_city_sf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11306 entries, 46 to 162491
Data columns (total 100 columns):
Unnamed: 0                                  11306 non-null int64
activity_year                               11306 non-null int64
lei                                         11306 non-null object
derived_msa_md                              11306 non-null int64
state_code                                  11306 non-null object
county_code                                 11306 non-null object
census_tract                                11306 non-null object
conforming_loan_limit                       11306 non-null object
derived_loan_product_type                   11306 non-null object
derived_dwelling_category                   11306 non-null object
derived_ethnicity                           11306 non-null object
derived_race                                11306 non-null object
derived_sex                                 11306 non-null object
action_taken                     

In [162]:
### How many single-family home loans were made in the city of St. Louis?

stl_city_sf_made = stl_city_sf.loc[stl_city_sf['action_taken'].isin(orig)]
stl_city_sf_made.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5252 entries, 84 to 162491
Data columns (total 100 columns):
Unnamed: 0                                  5252 non-null int64
activity_year                               5252 non-null int64
lei                                         5252 non-null object
derived_msa_md                              5252 non-null int64
state_code                                  5252 non-null object
county_code                                 5252 non-null object
census_tract                                5252 non-null object
conforming_loan_limit                       5252 non-null object
derived_loan_product_type                   5252 non-null object
derived_dwelling_category                   5252 non-null object
derived_ethnicity                           5252 non-null object
derived_race                                5252 non-null object
derived_sex                                 5252 non-null object
action_taken                                525

In [156]:
# StL Metro single-family home mortgage data

stl_sf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136202 entries, 0 to 502571
Data columns (total 100 columns):
Unnamed: 0                                  136202 non-null int64
activity_year                               136202 non-null int64
lei                                         136202 non-null object
derived_msa_md                              136202 non-null int64
state_code                                  136202 non-null object
county_code                                 136198 non-null object
census_tract                                136202 non-null object
conforming_loan_limit                       136202 non-null object
derived_loan_product_type                   136202 non-null object
derived_dwelling_category                   136202 non-null object
derived_ethnicity                           136202 non-null object
derived_race                                136202 non-null object
derived_sex                                 136202 non-null object
action_taken        

In [130]:
stl_frames.county_code.unique()

array([29099.0, 29189.0, 29183.0, 29510.0, 29071.0, 29113.0, 29219.0,
       17163.0, 17005.0, '29071', '29189', '29113', '29183', '29099',
       '29510', '29219', nan, 17133.0, 17119.0, 17117.0, 17027.0, 17083.0,
       17013.0], dtype=object)

In [122]:
stl_frames.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136876 entries, 0 to 502571
Data columns (total 100 columns):
Unnamed: 0                                  136876 non-null int64
activity_year                               136876 non-null int64
lei                                         136876 non-null object
derived_msa_md                              136876 non-null int64
state_code                                  136876 non-null object
county_code                                 136872 non-null object
census_tract                                136876 non-null object
conforming_loan_limit                       136202 non-null object
derived_loan_product_type                   136876 non-null object
derived_dwelling_category                   136876 non-null object
derived_ethnicity                           136876 non-null object
derived_race                                136876 non-null object
derived_sex                                 136876 non-null object
action_taken        

In [100]:
### Filtering the larger dataframes into just the states we are interested in

df31_mo = df31.loc[df31['state_code'].isin(mo)]
df31_il = df31.loc[df31['state_code'].isin(il)]

In [101]:
### Exporting each state frame to a .csv

df31_mo.to_csv('./data/public_lar/mo/mo31.csv')
df31_il.to_csv('./data/public_lar/il/il31.csv')

In [ ]:
###############################################

In [ ]:
frame = pd.concat(df1, df2)

In [9]:
d1_mo = d1.loc[d1['state_code'].isin(mo)]

In [15]:
d1_stl = d1.loc[d1['derived_msa_md'].isin(stl)]

In [ ]:
d1_stl = d1.loc[d1['derived_msa_md'].isin(stl)]

In [24]:
# To preview data 

row_count = 5
chunks = []
for chunk in pd.read_csv('./data/2018_public_lar_csv.csv', chunksize=row_count): 
    print chunk.head()
    # yeschunk = ['derived_msa_md']=='41180'

   activity_year                   lei  derived_msa_md state_code  \
0           2018  549300HW662MN1WU8550           40140         CA   
1           2018  549300HW662MN1WU8550           33460         MN   
2           2018  549300HW662MN1WU8550           47664         MI   
3           2018  549300HW662MN1WU8550           12060         GA   
4           2018  549300HW662MN1WU8550           29820         NV   

   county_code  census_tract conforming_loan_limit derived_loan_product_type  \
0         6071    6071009709                     C            FHA:First Lien   
1        27141   27141030101                     C   Conventional:First Lien   
2        26125   26125183900                     C   Conventional:First Lien   
3        13057   13057090400                     C   Conventional:First Lien   
4        32003   32003005853                     C   Conventional:First Lien   

              derived_dwelling_category       derived_ethnicity  ...  \
0  Single Family (1-4 Units):Sit

    activity_year                   lei  derived_msa_md state_code  \
20           2018  549300HW662MN1WU8550           27340         NC   
21           2018  549300HW662MN1WU8550           47664         MI   
22           2018  549300HW662MN1WU8550           47664         MI   
23           2018  549300HW662MN1WU8550           16974         IL   
24           2018  549300HW662MN1WU8550           40140         CA   

    county_code  census_tract conforming_loan_limit derived_loan_product_type  \
20        37133   37133002201                     C            FHA:First Lien   
21        26125   26125128900                     C   Conventional:First Lien   
22        26125   26125145400                     C   Conventional:First Lien   
23        17031   17031808002                     C   Conventional:First Lien   
24         6065    6065046403                     C   Conventional:First Lien   

               derived_dwelling_category       derived_ethnicity  ...  \
20  Single Family (

    activity_year                   lei  derived_msa_md state_code  \
40           2018  549300HW662MN1WU8550           29180         LA   
41           2018  549300HW662MN1WU8550           48424         FL   
42           2018  549300HW662MN1WU8550           27260         FL   
43           2018  549300HW662MN1WU8550           29460         FL   
44           2018  549300HW662MN1WU8550           31084         CA   

    county_code  census_tract conforming_loan_limit derived_loan_product_type  \
40        22055   22055002103                     C            FHA:First Lien   
41        12099   12099005814                     C            FHA:First Lien   
42        12109   12109020804                     C   Conventional:First Lien   
43        12105   12105011902                     C   Conventional:First Lien   
44         6037    6037236100                     C   Conventional:First Lien   

               derived_dwelling_category        derived_ethnicity  ...  \
40  Single Family 

    activity_year                   lei  derived_msa_md state_code  \
60           2018  549300HW662MN1WU8550           35614         NJ   
61           2018  549300HW662MN1WU8550           11244         CA   
62           2018  549300HW662MN1WU8550           36740         FL   
63           2018  549300HW662MN1WU8550           38300         PA   
64           2018  549300HW662MN1WU8550           36740         FL   

    county_code  census_tract conforming_loan_limit derived_loan_product_type  \
60        34017   34017006100                     C   Conventional:First Lien   
61         6059    6059063903                     C   Conventional:First Lien   
62        12097   12097043204                     C             VA:First Lien   
63        42003   42003413201                     C   Conventional:First Lien   
64        12095   12095017804                     C            FHA:First Lien   

               derived_dwelling_category       derived_ethnicity  ...  \
60  Single Family (

    activity_year                   lei  derived_msa_md state_code  \
80           2018  549300HW662MN1WU8550           12580         MD   
81           2018  549300HW662MN1WU8550           31140         KY   
82           2018  549300HW662MN1WU8550           36260         UT   
83           2018  549300HW662MN1WU8550           30860         UT   
84           2018  549300HW662MN1WU8550           30860         UT   

    county_code  census_tract conforming_loan_limit derived_loan_product_type  \
80        24005   24005400400                     C   Conventional:First Lien   
81        21111   21111011110                     C   Conventional:First Lien   
82        49011   49011126002                     C   Conventional:First Lien   
83        49005   49005000202                     C   Conventional:First Lien   
84        49005   49005000202                     C   Conventional:First Lien   

               derived_dwelling_category        derived_ethnicity  ...  \
80  Single Family 

...

     activity_year                   lei  derived_msa_md state_code  \
155           2018  549300HW662MN1WU8550           22744         FL   
156           2018  549300HW662MN1WU8550           36260         UT   
157           2018  549300HW662MN1WU8550           16974         IL   
158           2018  549300HW662MN1WU8550           19740         CO   
159           2018  549300HW662MN1WU8550           12060         GA   

     county_code  census_tract conforming_loan_limit  \
155        12011   12011043100                     C   
156        49011   49011126801                     C   
157        17031   17031081403                     C   
158         8005    8005007302                     C   
159        13067   13067030310                     C   

    derived_loan_product_type             derived_dwelling_category  \
155   Conventional:First Lien  Single Family (1-4 Units):Site-Built   
156   Conventional:First Lien  Single Family (1-4 Units):Site-Built   
157   Conventional:Firs

     activity_year                   lei  derived_msa_md state_code  \
170           2018  549300HW662MN1WU8550           10580         NY   
171           2018  549300HW662MN1WU8550           37964         PA   
172           2018  549300HW662MN1WU8550           24540         CO   
173           2018  549300HW662MN1WU8550           33124         FL   
174           2018  549300HW662MN1WU8550           11460         MI   

     county_code  census_tract conforming_loan_limit  \
170        36083   36083051600                     C   
171        42101   42101003702                     C   
172         8123    8123001005                     C   
173        12086   12086016900                     C   
174        26161   26161407600                     C   

    derived_loan_product_type             derived_dwelling_category  \
170            FHA:First Lien  Single Family (1-4 Units):Site-Built   
171            FHA:First Lien  Single Family (1-4 Units):Site-Built   
172   Conventional:Firs

     activity_year                   lei  derived_msa_md state_code  \
190           2018  549300HW662MN1WU8550           38060         AZ   
191           2018  549300HW662MN1WU8550           38940         FL   
192           2018  549300HW662MN1WU8550           48300         WA   
193           2018  549300HW662MN1WU8550           38060         AZ   
194           2018  549300HW662MN1WU8550           35614         NY   

     county_code  census_tract conforming_loan_limit  \
190         4013    4013420214                     C   
191        12111   12111380400                     C   
192        53017   53017950700                     C   
193         4013    4013420210                     C   
194        36081   36081137700                     C   

    derived_loan_product_type             derived_dwelling_category  \
190            FHA:First Lien  Single Family (1-4 Units):Site-Built   
191   Conventional:First Lien  Single Family (1-4 Units):Site-Built   
192            FHA:Firs

     activity_year                   lei  derived_msa_md state_code  \
210           2018  549300HW662MN1WU8550           44140         MA   
211           2018  549300HW662MN1WU8550           99999         NE   
212           2018  549300HW662MN1WU8550           29340         LA   
213           2018  549300HW662MN1WU8550           15380         NY   
214           2018  549300HW662MN1WU8550           45540         FL   

     county_code  census_tract conforming_loan_limit  \
210        25013   25013811000                     C   
211        31053   31053963800                     C   
212        22019   22019001000                     C   
213        36029   36029000800                     C   
214        12119   12119911200                     C   

    derived_loan_product_type             derived_dwelling_category  \
210   Conventional:First Lien  Single Family (1-4 Units):Site-Built   
211            FHA:First Lien  Single Family (1-4 Units):Site-Built   
212   Conventional:Firs

KeyboardInterrupt: 

In [2]:
# To split into smaller csvs of 500,000

divisor = 500000

outfileno = 1
outfile = None

with open('./data/2018_public_lar_csv.csv', 'r') as infile:
    for index, row in enumerate(csv.reader(infile)):
        if index % divisor == 0:
            if outfile is not None:
                outfile.close()
            outfilename = './data/public_lar_2018-{}.csv'.format(outfileno)
            outfile = open(outfilename, 'w')
            outfileno += 1
            writer = csv.writer(outfile)
        writer.writerow(row)

In [4]:
### Reading in public_lar csv files one at a time and filtering to get only Missouri/Illinois data

# df31 = pd.read_csv('./data/public_lar/public_lar_2018-31.csv', names=names, low_memory=False)


### Combines all public_lar files nationwide and loads into a huge df. Takes forever. 

# path = './data/public_lar/'
# all_files = glob.glob(path + "/*.csv")

# li = []

# for filename in all_files:
    # df = pd.read_csv(filename, index_col=None, header=0, low_memory=False)
    # li.append(df)

# frame = pd.concat(li, axis=0, ignore_index=True)

### Combines separate Missouri/Illinois loan app files generated from prev processing
### Combines into a large dataframe for each state.
### Takes a little less time. 

# test_path = './data/public_lar/test/'
mo_path = './data/public_lar/mo/'
il_path = './data/public_lar/il/'
# mo_files = glob.glob(mo_path + "/*.csv")
# il_files = glob.glob(il_path + "/*.csv")

# for filename in mo_files:
    # df = pd.read_csv(filename, index_col=None, low_memory=False)
    # mo_list.append(df)

# for filename in il_files:
    # df = pd.read_csv(filename, index_col=None, low_memory=False)
    # il_list.append(df)

# mo_frame = pd.concat(mo_list, axis=0, ignore_index=True)
# il_frame = pd.concat(il_list, axis=0, ignore_index=True)

In [5]:
### Exporting each state's loan application data into its own file

# mo_frame.to_csv('./data/public_lar/missouri.csv')
# il_frame.to_csv('./data/public_lar/illinois.csv')

### Loading the public_lar files for Missouri and Illinois, adding each df to a list, and concatenating all frames 
### So there is one df for each state

mo_filepath = ('./data/public_lar/mo/*.csv')
il_filepath = ('./data/public_lar/il/*.csv')

my_list_mo = []
my_list_il = []

final_mo = []
final_il = []

# for name in glob.glob(mo_filepath):
    # my_list_mo.append(name)
    
# for name in glob.glob(il_filepath):
    # my_list_il.append(name)

# for file in my_list_mo:
   # temp = pd.read_csv(file, engine='python')
# final_mo.append(temp)

# for file in my_list_il:
    # temp = pd.read_csv(file, engine='python')
# final_il.append(temp)

# mo_frame = pd.read_csv('./data/public_lar/mo/missouri.csv', low_memory = False)
# il_frame = pd.read_csv('./data/public_lar/il/illinois.csv', low_memory = False)